In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import graphviz
from datetime import datetime

from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import export_graphviz
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
import warnings


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [2]:
dataset = pd.read_csv("nba2021.csv")
print("dataset.keys(): {}".format(dataset.keys()))
X = dataset.drop(['Player', 'Pos', 'Age', 'Tm', 'GS', 'MP', 'FGA', '3PA', '2PA', 'FTA', 'ORB','DRB'], axis=1)
Y = dataset['Pos']


#Split
x_train, x_test, y_train, y_test = train_test_split(X,Y, stratify=dataset['Pos'], random_state=0, test_size=0.25)

#standardization
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)
X = scaler.fit_transform(X)

dataset.keys(): Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')


In [10]:
def create_and_train_lstm_model(X_train, y_train, log_dir='Logs', epochs=200):

    # Create Sequential model
    model = Sequential()
    # model.add(LSTM(128, return_sequences=True, activation='relu', input_shape=(30, 126)))
    # model.add(LSTM(256, return_sequences=True, activation='relu'))
    # model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu', input_shape=(12, 1)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(5, activation='softmax'))

    # Compile the model
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

    # Train the model
    model.fit(X_train, y_train, epochs=epochs)

    return model

In [6]:
def calculate_accuracy_and_confusion_matrix(model, X_test, y_test):
    # Step 1: Predict using the model
    yhat = model.predict(X_test)

    # Step 2: Convert predictions and true labels to lists
    ytrue = np.argmax(y_test, axis=1).tolist()
    yhat = np.argmax(yhat, axis=1).tolist()

    # Step 3: Calculate confusion matrix
    confusion_matrix = multilabel_confusion_matrix(ytrue, yhat)

    # Step 4: Calculate accuracy score
    accuracy = accuracy_score(ytrue, yhat)

    return accuracy, confusion_matrix

In [11]:
create_and_train_lstm_model(x_train, y_train)

Epoch 1/200


ValueError: in user code:

    File "c:\Users\jbsha\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\jbsha\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\jbsha\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\jbsha\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1126, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\jbsha\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
   